In [ ]:
import os
import numpy as np
import pandas as pd
import geopandas as gp
from shapely.geometry import Point, Polygon
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
datapath = Path('../data/geopandas/')

## TEST YOUR SKILLS #0
- make a new geodataframe of the parks
- add a columns with centroids for each park
- plot an interactive window with the park centroids and the neighborhoods
- hints: 
    - remember the shapely methods are available for each geometry object (e.g. `centroid()`) 
    - you can loop over the column in a couple different ways
    - you can define which columns contains the geometry of a geodataframe
    - you will likely have to define the CRS

In [ ]:
parks = gp.read_file(datapath / 'Madison_parks.geojson')
hoods = gp.read_file(datapath / 'Neighborhood_Associations.geojson')

In [ ]:
# loopy solution
parks_cent = parks.copy()
centroids = []
for i in parks_cent.geometry.values:
    centroids.append(i.centroid)
parks_cent['centroid'] = centroids

In [ ]:
# do it all at once with a list comprehension
parks_cent['centroid'] = [i.centroid for i in parks_cent.geometry]

In [ ]:
# set the geometry and CRS
parks_cent.set_geometry('centroid', inplace=True)
parks_cent.set_crs(parks.crs, inplace=True);

In [ ]:
m_hoods = hoods.explore()
parks_cent.explore(m=m_hoods)

# TEST YOUR SKILLS #1
Using the `bounds` geodataframe you just made, write a function to visualize predicate behaviors.
- your function should accept a left geodataframe, a right geodataframe, and a string for the predicate
- your function should plot:
    - the left geodataframe in (default) blue
    - the result of the spatial join operation in another color
    - the right geodataframe in another color with outline only
- then you should set the title of the plot to the string predicate value used
- the geodataframes to test with are `isthmus_parks` and `bounds`
- your function should `return` the joined geodataframe

- a couple hints:
    - in the `plot` method are a couple args called `facecolor` and `edgecolor` that will help plot the rectangle
    - there are other predicates to try out 

- _advanced options_: if that was easy, you can try a couple other things like:
    - explore joins with points and lines in addition to just polygons
    - change around the `bounds` polygon dimensions 
    - use `explore()` to make an interactive map

### first have to bring over some things from the main lesson

In [ ]:
parks.to_crs(3071, inplace=True)
hoods.to_crs(parks.crs, inplace=True)
isthmus = hoods.loc[hoods['NEIGHB_NAME'].str.contains('Marquette') | 
                   hoods['NEIGHB_NAME'].str.contains('Tenney')]
from shapely.geometry import box
bbox = box(570600, 290000, 573100, 291700)
bounds = gp.GeoDataFrame(geometry=[bbox],crs=parks.crs)
isthmus_parks = gp.sjoin(left_df=parks, right_df=isthmus)
isthmus_parks.drop(columns=[ 'index_right','OBJECTID_right', 'NA_ID', 'STATUS', 'CLASSIFICA', 'Web',
       'ShapeSTArea', 'ShapeSTLength'], inplace=True)

In [ ]:
def show_predicate(ldf,rdf,predicate):
    sj = gp.sjoin(ldf, rdf, predicate=predicate)
    ax = ldf.plot()
    sj.plot(ax=ax, color='black')
    rdf.plot(facecolor='none', edgecolor='orange', ax=ax)
    ax.set_title(predicate)
    return sj

In [ ]:
sj = show_predicate(isthmus_parks, bounds, 'intersects')
sj.head()

In [ ]:
sj = show_predicate(bounds, isthmus_parks, 'within')
sj.head()

# TEST YOUR SKILLS _OPTIONAL_
We have an Excel file that contains a crosswalk between SPECIES number as provided and species name. Can we bring that into our dataset and evaluate some conclusions about tree species by neighborhood?
- start with the `trees_with_hoods` GeoDataFrame
- load up and join the data from datapath / 'Madison_Tree_Species_Lookup.xlsx'
- hint: check the dtypes before merging - if you are going to join on a column, the column must be the same dtype in both dataframes
- Make a multipage PDF with a page for each neighborhood showing a bar chart of the top ten tree species (by name) in each neighborhood
- Make a map (use explore, or save to SHP or geojson) showing the neighborhoods with a color-coded field showing the most common tree species for each neighborhood

You will need a few pandas operations that we have only touched on a bit:  

`groupby`, `count`, `merge`, `read_excel`, `sort_values`, `iloc`

In [ ]:
# read back in the trees and hoods data
trees = gp.read_file(datapath / 'Street_Trees.geojson', index_col=0)
trees.to_crs(hoods.crs, inplace=True)
trees_with_hoods = trees[['SPECIES','DIAMETER','geometry']].sjoin(hoods[['NEIGHB_NAME','geometry']])
trees_with_hoods.head()

In [ ]:
# now read the excel file with tree species lookup - might need to fiddle with skiprows parameter
tree_species = pd.read_excel(datapath / 'Madison_Tree_Species_Lookup.xlsx', 
                             skiprows = 6)
tree_species

In [ ]:
# check out the data types 
tree_species.dtypes

In [ ]:
trees_with_hoods.dtypes

In [ ]:
# d'oh! Code in tree_species and SPECIES in trees_with_hoods are different types.
# To make them consistent, let's convert SPECIES in trees_with_hoods to int
trees_with_hoods.SPECIES = [int(i) for i in trees_with_hoods.SPECIES]

In [ ]:
trees_with_hoods.dtypes

In [ ]:
# now we can merge - check out the left_on, right_on args
trees_final = trees_with_hoods.merge(tree_species, left_on='SPECIES', right_on='Code')
trees_final

In [ ]:
# now the multipage plots
with PdfPages(datapath / 'TreePlots.pdf') as outpdf:
    # first groupby neighborhoods 
    for cn, cg in trees_final.groupby('NEIGHB_NAME'):
        #then, for each neighborhood, group by "Description" to get counts by name
        counts = cg.groupby('Description')['SPECIES'].count()
        # sort them in reverse value
        counts.sort_values(ascending=False, inplace=True)
        #make a bar chart of the top ten counts
        counts[:10].plot.bar()
        # set up a title
        plt.title(f'top 10 trees in {cn}')
        # when the x-axis labels are long they can get cut off. tight_layout can help
        plt.tight_layout()
        outpdf.savefig()
        plt.close('all')

In [ ]:
# we can do this in an extra-pythonic way as well, chaining operations together
# advantage is it's faster to run but can be harder to initially understand and debug
with PdfPages(datapath / 'TreePlots.pdf') as outpdf:
    # first groupby neighborhoods 
    for cn, cg in trees_final.groupby('NEIGHB_NAME'):
        cg.groupby('Description')['SPECIES'].count().sort_values(ascending=False)[:10].plot.bar()
        # set up a title
        plt.title(f'top 10 trees in {cn}')
        # when the x-axis labels are long they can get cut off. tight_layout can help
        plt.tight_layout()
        outpdf.savefig()
        plt.close('all')

### Now let's find the most common tree species in each neighborhood and make a map. There are some ["sophisticated" ways](https://stackoverflow.com/questions/52243060/get-row-value-of-maximum-count-after-applying-group-by-in-pandas) using lots of pandas intrinsic functionality that can work, but we can also do it in a few (hopefully) logical explicit steps. 

In [ ]:
# we can make a couple empty lists and just append the neighborhood name and the index of the 
# maximum occuring tree species in each in a loop. Still some "cleverness"
hood = []
max_tree = []
for cn, cg in trees_final.groupby('NEIGHB_NAME'):
    hood.append(cn)
    max_tree.append(cg.groupby('Description').count().sort_values(by='SPECIES', ascending=False).iloc[0].name)

In [ ]:
# make a dataframe with these values
mts = pd.DataFrame(index=hood, data={'max_tree':max_tree})

In [ ]:
#now join this back into the GeoDataFrame of hoods
hoods.merge(mts, left_on='NEIGHB_NAME', right_index=True)[['NEIGHB_NAME','max_tree', 'geometry']].explore(column='max_tree')